In [2]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [3]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A100-SXM4-80GB


#### Import Graph500_Scale23_EdgeFactor64

In [4]:
# Import dataset from saved PyTorch dataset
data = torch.load("/mnt/ephemeral/gnn/dataset/Graph500/graph500_scale23_ef64.pt")

In [ ]:
# Transform dense edge index to sparse adjacency matrix
import torch_geometric.transforms as T

data = T.ToSparseTensor()(data)
data = data.pin_memory()

#### Graph Information

In [8]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[8388608, 64], y=[8388608], train_mask=[8388608], adj_t=[8388608, 8388608, nnz=517557419])


In [9]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([8388608, 64])
tensor([[-0.5417, -1.8515,  0.5649,  ...,  1.2969, -1.7191, -1.0602],
        [ 1.2967,  0.8535, -0.9299,  ..., -1.5634, -0.5079,  2.0238],
        [ 0.4188, -1.6258, -1.6195,  ...,  0.4586,  0.9378, -1.8361],
        ...,
        [-2.0960,  0.8987,  0.8080,  ...,  2.0975, -2.0398, -0.2668],
        [-0.5311,  2.2089,  1.0266,  ..., -0.5919, -2.4156,  0.8955],
        [ 1.1232,  0.1070, -1.9571,  ..., -1.6113, -0.8580, -1.8450]])


In [10]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([      0,       0,       0,  ..., 8388607, 8388607, 8388607]),
             col=tensor([      0,       1,       2,  ..., 1320822, 6800756, 8065975]),
             size=(8388608, 8388608), nnz=517557419, density=0.00%)


In [11]:
# Density of adjacency matrix
print(data.adj_t.density()*100)

0.0007354933288183929


In [12]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([8388608])
tensor([ 5, 11, 32,  ..., 26, 48, 10])


In [13]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([8388608])
tensor([False, False,  True,  ..., False, False, False])


#### Single-layer GCN

In [14]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(64, 64)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01)

    def forward(self, x, adj_t):
        x = self.gcn1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

#### To get GFLOPS result

In [15]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        #acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        #if(epoch % 10 == 0):
            #print('Epoch:', epoch)
            #print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

In [16]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
start.record()
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(64, 64)
)

Elapsed Time (100 Epochs): 377.13146875 seconds


#### To get accuracy result

In [12]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        acc = accuracy(out.argmax(dim=1), data.y)
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

In [13]:
# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))

GCN(
  (gcn1): GCNConv(64, 64)
)

Epoch   0 | Train Loss: 4.490 | Train Acc:   1.56%
Epoch  10 | Train Loss: 4.217 | Train Acc:   1.57%
Epoch  20 | Train Loss: 4.165 | Train Acc:   1.57%
Epoch  30 | Train Loss: 4.165 | Train Acc:   1.57%
Epoch  40 | Train Loss: 4.161 | Train Acc:   1.59%
Epoch  50 | Train Loss: 4.160 | Train Acc:   1.60%
Epoch  60 | Train Loss: 4.160 | Train Acc:   1.61%
Epoch  70 | Train Loss: 4.160 | Train Acc:   1.62%
Epoch  80 | Train Loss: 4.160 | Train Acc:   1.63%
Epoch  90 | Train Loss: 4.160 | Train Acc:   1.63%
Epoch 100 | Train Loss: 4.160 | Train Acc:   1.63%
